# 第二章 高级的RAG管道

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

载入文本数据

In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["data/人工智能.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

## 一、基础RAG通道

这里通过将 documents 中各个文档的文本连接成一个字符串，然后创建了一个 Document 实例，该实例代表了整个文档集合。

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
# 将中文标点符号替换成英文标点符号，方便后续处理
# 如果是英文文档，可以跳过这一步
# 不处理的话，会导致无法正确切分中文句子，会影响后续sentence_window的大小，导致输入长度大于gpt-3.5-turbo的最大限制
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')

llm-使用 OpenAI 类创建了一个 GPT-3.5-turbo 模型的实例，并设置了温度参数为 0.1。  
service_context-使用 ServiceContext 类创建了一个服务上下文实例，包含了前面创建的 GPT-3.5-turbo 模型以及指定的嵌入模型。  
index-使用 VectorStoreIndex.from_documents 方法，基于之前创建的文档和服务上下文，创建了一个向量存储索引。

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-zh-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

将之前创建的向量存储索引转换为查询引擎，以便后续进行查询操作。

In [6]:
query_engine = index.as_query_engine()

使用查询引擎执行了一个查询操作，查询给定的问题。

In [7]:
response = query_engine.query(
    "在寻找项目以积累经验时应采取哪些步骤?"
)
print(str(response))

## 二、使用Trulens进行评测

In [8]:
eval_questions = []
with open('data/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

加上自定义问题。

In [9]:
# You can try your own question:
new_question = "什么是适合我的人工智能工作?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

通过调用 reset_database() 方法重置 Trulens 数据库。清空之前的记录和反馈数据。

In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

使用 get_prebuilt_trulens_recorder 函数创建一个 Trulens 记录器 (tru_recorder)，该记录器与给定的查询引擎 (query_engine) 相关联。同时，指定了应用程序的标识为 "Direct Query Engine"。

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

使用 tru_recorder 记录器开始记录过程，遍历 eval_questions 列表，对每个问题进行查询，并将查询引擎的响应记录下来。

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

获取 Trulens 记录和反馈数据。用于后续分析和评估。

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

运行 Trulens 仪表板以可视化评估结果。

In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

## 三、高级的RAG通道

### 3.1 滑窗句子检索

创建 OpenAI 的 GPT-3.5-turbo 语言模型实例：

In [34]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

使用辅助函数 build_sentence_window_index 创建基于窗口的句子索引：

In [35]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="sentence_index"
)

使用辅助函数 get_sentence_window_query_engine 获取基于句子窗口的查询引擎：

In [36]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

对一个特定的问题进行查询并打印结果：

In [37]:
window_response = sentence_window_engine.query(
    "如何开始人工智能个人项目?"
)
print(str(window_response))

重置 Trulens 数据库，  
使用 Trulens 记录器对基于窗口的句子索引进行评估，记录查询结果：

In [40]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [41]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

获取性能评估的排行榜：

In [42]:
tru.get_leaderboard(app_ids=[])

In [43]:
# launches on http://localhost:8501/
tru.run_dashboard()

### 3.2 自动合并检索

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "如何开始人工智能个人项目?"
)
print(str(auto_merging_response))

In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

In [30]:
tru.get_leaderboard(app_ids=[])

In [33]:
# launches on http://localhost:8501/
tru.run_dashboard()